# Installs

In [ ]:
!pip install wandb -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 36.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.5/199.5 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 14.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 2.4 MB/s eta 0:00:00


### Levenshtein

This may take a while

In [ ]:
!pip install wandb --quiet
!pip install python-Levenshtein -q
!git clone --recursive https://github.com/parlance/ctcdecode.git
!pip install wget -q
%cd ctcdecode
!pip install . -q
%cd ..

!pip install torchsummaryX -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.5/175.5 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 81.3 MB/s eta 0:00:00
Cloning into 'ctcdecode'...
remote: Enumerating objects: 1102, done.
remote: Counting objects: 100% (39/39), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 1102 (delta 16), reused 32 (delta 14), pack-reused 1063
Receiving objects: 100% (1102/1102), 782.27 KiB | 922.00 KiB/s, done.
Resolving deltas: 100% (529/529), done.
Submodule 'third_party/ThreadPool' (https://github.com/progschj/ThreadPool.git) registered for path 'third_party/ThreadPool'
Submodule 'third_party/kenlm' (https://github.com/kpu/kenlm.git) registered for path 'third_party/kenlm'
Cloning into '/content/ctcdecode/third_party/ThreadPool'...
remote: Enumerating objects: 82, done.        
remote: Total 82 (delta 0), reused 0 (delta 0), pack-reused 82        
Cloning into '/content/ctcdecode/third_party/kenlm'...
remote: Enumerating objects: 1414

## Imports

In [ ]:
import torch
import random
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from torchsummaryX import summary
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence

import torchaudio.transforms as tat

from sklearn.metrics import accuracy_score
import gc

import zipfile
import pandas as pd
from tqdm import tqdm
import os
import datetime

# imports for decoding and distance calculation
import ctcdecode
import Levenshtein
from ctcdecode import CTCBeamDecoder

import warnings
warnings.filterwarnings('ignore')

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device: ", device)

Device:  cuda


# Kaggle Setup

In [ ]:
!pip install --upgrade --force-reinstall --no-deps kaggle==1.5.8
!mkdir /root/.kaggle

with open("/root/.kaggle/kaggle.json", "w+") as f:
    f.write('{"username":"jasongao0917","key":"9aaf0189605045a47217ffc7b87d06fc"}') # TODO: Put your kaggle username & key here

!chmod 600 /root/.kaggle/kaggle.json

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 7.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for kaggle: filename=kaggle-1.5.8-py3-none-any.whl size=73272 sha256=012d143883080098ba40461e1104561eb0951b14b3f01800631ea121b1716f2a
  Stored in directory: /root/.cache/pip/wheels/d4/02/ef/3f8c8d86b8d5388a1d3155876837f1a1a3143ab3fc2ff1ffad
Successfully built kaggle
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.5.13
    Uninstalling kaggle-1.5.13:
      Successfully uninstalled kaggle-1.5.13


In [ ]:
!kaggle competitions download -c 11-785-s23-hw3p2

100% 15.9G/15.9G [14:25<00:00, 23.8MB/s]
100% 15.9G/15.9G [14:25<00:00, 19.8MB/s]


In [ ]:
'''
This will take a couple minutes, but you should see at least the following:
11-785-f22-hw3p2.zip  ctcdecode  hw3p2
'''
!unzip -q 11-785-s23-hw3p2.zip
!ls

11-785-s23-hw3p2  11-785-s23-hw3p2.zip	ctcdecode  sample_data


# Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


# Dataset and Dataloader

In [ ]:
# ARPABET PHONEME MAPPING
# DO NOT CHANGE
# This overwrites the phonetics.py file.

CMUdict_ARPAbet = {
    "" : " ",
    "[SIL]": "-", "NG": "G", "F" : "f", "M" : "m", "AE": "@",
    "R"    : "r", "UW": "u", "N" : "n", "IY": "i", "AW": "W",
    "V"    : "v", "UH": "U", "OW": "o", "AA": "a", "ER": "R",
    "HH"   : "h", "Z" : "z", "K" : "k", "CH": "C", "W" : "w",
    "EY"   : "e", "ZH": "Z", "T" : "t", "EH": "E", "Y" : "y",
    "AH"   : "A", "B" : "b", "P" : "p", "TH": "T", "DH": "D",
    "AO"   : "c", "G" : "g", "L" : "l", "JH": "j", "OY": "O",
    "SH"   : "S", "D" : "d", "AY": "Y", "S" : "s", "IH": "I",
    "[SOS]": "[SOS]", "[EOS]": "[EOS]"
}

CMUdict = list(CMUdict_ARPAbet.keys())
ARPAbet = list(CMUdict_ARPAbet.values())


PHONEMES = CMUdict[:-2]
LABELS = ARPAbet[:-2]

In [ ]:
# You might want to play around with the mapping as a sanity check here
print(len(PHONEMES))
print(len(LABELS))

41
41


### Train Data

In [ ]:
class AudioDataset(torch.utils.data.Dataset):

    # For this homework, we give you full flexibility to design your data set class.
    # Hint: The data from HW1 is very similar to this HW

    #TODO
    def __init__(self, root, partition= "train-clean-100"):
        '''
        Initializes the dataset.

        INPUTS: What inputs do you need here?
        '''

        # Load the directory and all files in them

        self.mfcc_dir = root + partition + '/mfcc' #TODO
        self.transcript_dir = root + partition + '/transcript' #TODO

        self.mfcc_files = sorted(os.listdir(self.mfcc_dir)) #TODO
        self.transcript_files = sorted(os.listdir(self.transcript_dir)) #TODO

        self.PHONEMES = PHONEMES

        #TODO
        # WHAT SHOULD THE LENGTH OF THE DATASET BE?
        self.length = len(self.mfcc_files)

        #TODO
        # HOW CAN WE REPRESENT PHONEMES? CAN WE CREATE A MAPPING FOR THEM?
        # HINT: TENSORS CANNOT STORE NON-NUMERICAL VALUES OR STRINGS
        tokens = range(len(PHONEMES))
        vocab = dict(zip(PHONEMES, tokens))

        #TODO
        # CREATE AN ARRAY OF ALL FEATUERS AND LABELS
        # WHAT NORMALIZATION TECHNIQUE DID YOU USE IN HW1? CAN WE USE IT HERE?
        '''
        You may decide to do this in __getitem__ if you wish.
        However, doing this here will make the __init__ function take the load of
        loading the data, and shift it away from training.
        '''
        self.mfccs = []
        self.transcripts = []

        for i in range(len(self.mfcc_files)):
            mfcc = np.load(self.mfcc_dir + '/' + self.mfcc_files[i])
            transcript = np.load(self.transcript_dir + '/' + self.transcript_files[i])
            # Cepstral Mean Normalization
            mfcc_mean = np.mean(mfcc, axis = 0)
            mfcc_std = np.std(mfcc, axis = 0)
            mfcc = (mfcc - mfcc_mean)/mfcc_std
            # Remove [SOS] and [EOS]
            transcript = transcript[1:-1]
            # Map phoneme (str) to index (int) values
            transcript = [vocab[label] for label in transcript]

            self.mfccs.append(mfcc)
            self.transcripts.append(transcript)


    def __len__(self):

        '''
        TODO: What do we return here?
        '''
        return self.length

    def __getitem__(self, ind):
        '''
        TODO: RETURN THE MFCC COEFFICIENTS AND ITS CORRESPONDING LABELS

        If you didn't do the loading and processing of the data in __init__,
        do that here.

        Once done, return a tuple of features and labels.
        '''

        mfcc = self.mfccs[ind] # TODO
        mfcc = torch.FloatTensor(mfcc)
        transcript = self.transcripts[ind] # TODO
        transcript = torch.tensor(transcript)

        return mfcc, transcript


    def collate_fn(self, batch):
        '''
        TODO:
        1.  Extract the features and labels from 'batch'
        2.  We will additionally need to pad both features and labels,
            look at pytorch's docs for pad_sequence
        3.  This is a good place to perform transforms, if you so wish.
            Performing them on batches will speed the process up a bit.
        4.  Return batch of features, labels, lenghts of features,
            and lengths of labels.
        '''
        # batch of input mfcc coefficients
        batch_mfcc = [mfcc[0] for mfcc in batch]  # TODO
        # batch of output phonemes
        batch_transcript = [mfcc[1] for mfcc in batch]  # TODO

        # HINT: CHECK OUT -> pad_sequence (imported above)
        # Also be sure to check the input format (batch_first)
        batch_mfcc_pad = pad_sequence(batch_mfcc, batch_first=True, padding_value=0) # TODO
        lengths_mfcc = [len(mfcc) for mfcc in batch_mfcc] # TODO

        batch_transcript_pad = pad_sequence(batch_transcript, batch_first=True, padding_value=0) # TODO
        lengths_transcript = [len(transcript) for transcript in batch_transcript] # TODO

        # You may apply some transformation, Time and Frequency masking, here in the collate function;
        # Food for thought -> Why are we applying the transformation here and not in the __getitem__?
        #                  -> Would we apply transformation on the validation set as well?
        #                  -> Is the order of axes / dimensions as expected for the transform functions?

        # Return the following values: padded features, padded labels, actual length of features, actual length of the labels
        return batch_mfcc_pad, batch_transcript_pad, torch.tensor(lengths_mfcc), torch.tensor(lengths_transcript)



### Test Data

In [ ]:
# Test Dataloader
#TODO
class AudioDatasetTest(torch.utils.data.Dataset):
  def __init__(self, root, partition = 'test-clean'):
        self.mfcc_dir = root + partition + '/mfcc'
        self.mfcc_files = sorted(os.listdir(self.mfcc_dir))
        self.length = len(self.mfcc_files)
        self.mfccs = []
        for i in range(len(self.mfcc_files)):
            mfcc = np.load(self.mfcc_dir + '/' + self.mfcc_files[i])
            # Cepstral Mean Normalization
            mfcc_mean = np.mean(mfcc, axis = 0)
            mfcc_std = np.std(mfcc, axis = 0)
            mfcc = (mfcc - mfcc_mean)/mfcc_std
            self.mfccs.append(mfcc)
            self.length = len(self.mfcc_files)

  def __len__(self):

        '''
        TODO: What do we return here?
        '''
        return self.length

  def __getitem__(self, ind):
        '''
        TODO: RETURN THE MFCC COEFFICIENTS AND ITS CORRESPONDING LABELS

        If you didn't do the loading and processing of the data in __init__,
        do that here.

        Once done, return a tuple of features and labels.
        '''

        mfcc = self.mfccs[ind] # TODO
        mfcc = torch.FloatTensor(mfcc)

        return mfcc

  def collate_fn(self, batch):
    batch_mfcc_pad = pad_sequence(batch, batch_first=True, padding_value=0)
    lengths_mfcc = [len(mfcc) for mfcc in batch]
    return batch_mfcc_pad, torch.tensor(lengths_mfcc)

### Data - Hyperparameters

In [ ]:
BATCH_SIZE = 64 # Increase if your device can handle it

transforms = [] # set of tranformations
# You may pass this as a parameter to the dataset class above
# This will help modularize your implementation

root = '/content/11-785-s23-hw3p2/'

### Data loaders

In [ ]:
# get me RAMMM!!!!
import gc
gc.collect()

0

In [ ]:
# Create objects for the dataset class
train_data = AudioDataset(root, partition='train-clean-360') #TODO
val_data = AudioDataset(root, partition='dev-clean') # TODO : You can either use the same class with some modifications or make a new one :)
test_data = AudioDatasetTest(root, partition='test-clean') #TODO

# Do NOT forget to pass in the collate function as parameter while creating the dataloader
train_loader =  torch.utils.data.DataLoader(
    dataset = train_data,
    num_workers = 4,
    batch_size = BATCH_SIZE,
    pin_memory = True,
    shuffle = True,
    collate_fn = train_data.collate_fn) #TODO

val_loader = torch.utils.data.DataLoader(
    dataset = val_data,
    num_workers = 2,
    batch_size = BATCH_SIZE,
    pin_memory = True,
    shuffle = False,
    collate_fn = val_data.collate_fn) #TODO

test_loader = torch.utils.data.DataLoader(
    dataset = test_data,
    num_workers = 2,
    batch_size = BATCH_SIZE,
    pin_memory = True,
    shuffle = False,
    collate_fn = test_data.collate_fn)  #TODO

print("Batch size: ", BATCH_SIZE)
print("Train dataset samples = {}, batches = {}".format(train_data.__len__(), len(train_loader)))
print("Val dataset samples = {}, batches = {}".format(val_data.__len__(), len(val_loader)))
print("Test dataset samples = {}, batches = {}".format(test_data.__len__(), len(test_loader)))

Batch size:  64
Train dataset samples = 104013, batches = 1626
Val dataset samples = 2703, batches = 43
Test dataset samples = 2620, batches = 41


In [ ]:
# sanity check
for data in train_loader:
    x, y, lx, ly = data
    print(x.shape, y.shape, lx.shape, ly.shape)
    break

torch.Size([64, 1695, 27]) torch.Size([64, 218]) torch.Size([64]) torch.Size([64])


# NETWORK

## Pyramid Bi-LSTM (pBLSTM)

In [ ]:
# Utils for network
torch.cuda.empty_cache()

class PermuteBlock(torch.nn.Module):
    def forward(self, x):
        return x.transpose(1, 2)

In [ ]:
class pBLSTM(torch.nn.Module):

    '''
    Pyramidal BiLSTM
    Read the write up/paper and understand the concepts and then write your implementation here.

    At each step,
    1. Pad your input if it is packed (Unpack it)
    2. Reduce the input length dimension by concatenating feature dimension
        (Tip: Write down the shapes and understand)
        (i) How should  you deal with odd/even length input?
        (ii) How should you deal with input length array (x_lens) after truncating the input?
    3. Pack your input
    4. Pass it into LSTM layer

    To make our implementation modular, we pass 1 layer at a time.
    '''

    def __init__(self, input_size, hidden_size):
        super(pBLSTM, self).__init__()

        self.blstm = nn.LSTM(input_size, hidden_size, 3, batch_first = True, bidirectional = True) # TODO: Initialize a single layer bidirectional LSTM with the given input_size and hidden_size

    def forward(self, x_packed): # x_packed is a PackedSequence

        # TODO: Pad Packed Sequence
        x, x_lengths = pad_packed_sequence(x_packed, batch_first=True)
        # Call self.trunc_reshape() which downsamples the time steps of x and increases the feature dimensions as mentioned above
        # self.trunc_reshape will return 2 outputs. What are they? Think about what quantites are changing.
        x, x_length = self.trunc_reshape(x, x_lengths)
        # TODO: Pack Padded Sequence. What output(s) would you get?
        x = pack_padded_sequence(x, x_length, batch_first=True, enforce_sorted=False)
        # TODO: Pass the sequence through bLSTM
        out, _ = self.blstm(x)
        # What do you return?
        return out

    def trunc_reshape(self, x, x_lens):
        # TODO: If you have odd number of timesteps, how can you handle it? (Hint: You can exclude them)
        ts_count = x.shape[1]
        if ts_count % 2 != 0:
          x = x[:, :-1, :]
        # TODO: Reshape x. When reshaping x, you have to reduce number of timesteps by a downsampling factor while increasing number of features by the same factor
        B, T, F = x.shape
        x = x.reshape(B, T//2, 2 * F)
        # TODO: Reduce lengths by the same downsampling factor
        x_lens //= 2
        return x, x_lens

# Encoder

In [ ]:
from torch.nn.modules.batchnorm import BatchNorm1d
class ResBlock(nn.Module):
  def __init__(self, cin, cout, stride = 1, downsample = False):
    super().__init__()
    self.conv1 = nn.Conv1d(cin, cout, kernel_size=3, stride = stride, bias = False, padding = 1)
    self.bn1 = nn.BatchNorm1d(cout)
    self.conv2 = nn.Conv1d(cout, cout, kernel_size=3, bias = False, padding = 1)
    self.bn2 = nn.BatchNorm1d(cout)
    self.downsample = nn.Sequential(
        nn.Conv1d(cin, cout, kernel_size=1, stride = stride, bias = False),
        nn.BatchNorm1d(cout)
        ) if downsample else None

  def forward(self, x: torch.Tensor):
    out = self.bn2(self.conv2(torch.relu(self.bn1(self.conv1(x)))))
    downsampled = self.downsample(x) if self.downsample is not None else x
    return torch.relu(out+downsampled)

In [ ]:
class ResNetC512L3(nn.Module):
  def __init__(self, layers, cin):
    super().__init__()
    self.conv = nn.Conv1d(cin, 512, kernel_size = 3, stride = 1, padding = 1, bias = False)
    self.bn = nn.BatchNorm1d(512)
    self.pool1 = nn.MaxPool1d(kernel_size=3, stride = 1, padding = 1)
    self.cin = 512
    residual_layers = [self.residual_layer(512, layers[0], 1),
                       self.residual_layer(1024, layers[1], 1),
                       self.residual_layer(256, layers[2], 1)]
    self.residual_layers = nn.Sequential(*residual_layers)

  def residual_layer(self, cout, num_block, stride):
    if num_block == 0:
      return nn.Identity()
    layers = [ResBlock(self.cin, cout, stride, stride > 1 or self.cin != cout)]
    self.cin = cout
    for _ in range(1, num_block):
      layers.append(ResBlock(self.cin, cout))
    return nn.Sequential(*layers)

  def forward(self, x):
    x = self.pool1(torch.relu(self.bn(self.conv(x))))
    x = self.residual_layers(x)
    return x

In [ ]:
from torch.autograd import Variable
class LockedDropout(nn.Module):
  def __init__(self):
    super().__init__()
  def forward(self, x, dropout=0.5):
    if not self.training or not dropout:
      return x
    m = x.data.new(1, x.size(1), x.size(2)).bernoulli_(1-dropout)
    mask = Variable(m, requires_grad=False)/(1-dropout)
    mask = mask.expand_as(x)
    return mask*x

In [ ]:
class Encoder(torch.nn.Module):
    '''
    The Encoder takes utterances as inputs and returns latent feature representations
    '''
    def __init__(self, input_size, encoder_hidden_size):
        super(Encoder, self).__init__()


        self.embedding = ResNetC512L3([2, 2, 2], input_size)#TODO: You can use CNNs as Embedding layer to extract features. Keep in mind the Input dimensions and expected dimension of Pytorch CNN.

         # How many pBLSTMs are required?
            # TODO: Fill this up with pBLSTMs - What should the input_size be?
            # Hint: You are downsampling timesteps by a factor of 2, upsampling features by a factor of 2 and the LSTM is bidirectional)
            # Optional: Dropout/Locked Dropout after each pBLSTM (Not needed for early submission)
            # https://github.com/salesforce/awd-lstm-lm/blob/dfd3cb0235d2caf2847a4d53e1cbd495b781b5d2/locked_dropout.py#L5
        self.RNN1 = pBLSTM(256*2, encoder_hidden_size)
        self.RNN2 = pBLSTM(encoder_hidden_size * 4, encoder_hidden_size)
        self.lockeddropout = LockedDropout()
        self.permuteBlock = PermuteBlock()


    def forward(self, x, x_lens):
        # Where are x and x_lens coming from? The dataloader
        #TODO: Call the embedding layer
        x = self.permuteBlock(x)
        x = self.embedding(x)
        x = self.permuteBlock(x)
        # TODO: Pack Padded Sequence
        # TODO: Pass Sequence through the pyramidal Bi-LSTM layer
        packed_x = pack_padded_sequence(x, x_lens, batch_first=True, enforce_sorted=False)
        packed_x = self.RNN1(packed_x)
        x, x_lens = pad_packed_sequence(packed_x, batch_first=True)
        packed_x = pack_padded_sequence(x, x_lens, batch_first=True, enforce_sorted=False)
        packed_x = self.RNN2(packed_x)
        x, x_lens = pad_packed_sequence(packed_x, batch_first=True)
        x = self.permuteBlock(x)
        x = self.lockeddropout(x, dropout=0.25)
        x = self.permuteBlock(x)
        # TODO: Pad Packed Sequence


        # Remember the number of output(s) each function returns

        return x, x_lens

# Decoder

In [ ]:
class Decoder(torch.nn.Module):

    def __init__(self, embed_size, output_size= 41):
        super().__init__()

        self.mlp = torch.nn.Sequential(
            PermuteBlock(), torch.nn.BatchNorm1d(2 * embed_size), PermuteBlock(),
            #TODO define your MLP arch. Refer HW1P2
            #Use Permute Block before and after BatchNorm1d() to match the size
            torch.nn.Linear(embed_size * 2, 2048),
            PermuteBlock(),
            torch.nn.BatchNorm1d(2048),
            PermuteBlock(),
            torch.nn.GELU(),
            torch.nn.Linear(2048, 2048),
            PermuteBlock(),
            torch.nn.BatchNorm1d(2048),
            PermuteBlock(),
            torch.nn.GELU(),
            torch.nn.Linear(2048, output_size)
        )

        self.softmax = torch.nn.LogSoftmax(dim=2)

    def forward(self, encoder_out):
        #TODO call your MLP
        out = self.mlp(encoder_out)
        #TODO Think what should be the final output of the decoder for the classification
        out = self.softmax(out)
        return out

In [ ]:
import torchaudio
class ASRModel(torch.nn.Module):

    def __init__(self, input_size, embed_size= 192, output_size= len(PHONEMES)):
        super().__init__()

        self.augmentations  = torch.nn.Sequential(
            #TODO Add Time Masking/ Frequency Masking
            #Hint: See how to use PermuteBlock() function defined above
            PermuteBlock(),
            torchaudio.transforms.FrequencyMasking(freq_mask_param=4),
            torchaudio.transforms.TimeMasking(time_mask_param=200),
            PermuteBlock()
        )
        self.encoder = Encoder(input_size, embed_size)       # TODO: Initialize Encoder
        self.decoder = Decoder(embed_size, output_size)       # TODO: Initialize Decoder



    def forward(self, x, lengths_x):

        if self.training:
            x = self.augmentations(x)

        encoder_out, encoder_lens   = self.encoder(x, lengths_x)
        decoder_out                 = self.decoder(encoder_out)

        return decoder_out, encoder_lens

## INIT ASR

In [ ]:
model = ASRModel(
    input_size  = 27,
    embed_size  = 256, #TODO
    output_size = len(PHONEMES)
).to(device)
print(model)
summary(model, x.to(device), lx)

ASRModel(
  (augmentations): Sequential(
    (0): PermuteBlock()
    (1): FrequencyMasking()
    (2): TimeMasking()
    (3): PermuteBlock()
  )
  (encoder): Encoder(
    (embedding): ResNetC512L3(
      (conv): Conv1d(27, 512, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
      (bn): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (pool1): MaxPool1d(kernel_size=3, stride=1, padding=1, dilation=1, ceil_mode=False)
      (residual_layers): Sequential(
        (0): Sequential(
          (0): ResBlock(
            (conv1): Conv1d(512, 512, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
            (bn1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (conv2): Conv1d(512, 512, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
            (bn2): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          )
          (1): ResBlock(
            (conv1)

,Kernel Shape,Output Shape,Params,Mult-Adds
Layer,,,,
0_augmentations.PermuteBlock_0,-,"[64, 27, 1695]",NaN,NaN
1_augmentations.FrequencyMasking_1,-,"[64, 27, 1695]",NaN,NaN
2_augmentations.TimeMasking_2,-,"[64, 27, 1695]",NaN,NaN
3_augmentations.PermuteBlock_3,-,"[64, 1695, 27]",NaN,NaN
4_encoder.PermuteBlock_permuteBlock,-,"[64, 27, 1695]",NaN,NaN
5_encoder.embedding.Conv1d_conv,"[27, 512, 3]","[64, 512, 1695]",41472.0,7.029504e+07
6_encoder.embedding.BatchNorm1d_bn,[512],"[64, 512, 1695]",1024.0,5.120000e+02
7_encoder.embedding.MaxPool1d_pool1,-,"[64, 512, 1695]",NaN,NaN
8_encoder.embedding.residual_layers.0.0.Conv1d_conv1,"[512, 512, 3]","[64, 512, 1695]",786432.0,1.333002e+09


# Training Config

In [ ]:
config = {
    "beam_width" : 3,
    "lr" : 2e-3,
    "epochs" : 60
    } # Feel free to add more items here

In [ ]:
#TODO


criterion = nn.CTCLoss()# Define CTC loss as the criterion. How would the losses be reduced?
# CTC Loss: https://pytorch.org/docs/stable/generated/torch.nn.CTCLoss.html
# Refer to the handout for hints

optimizer =  torch.optim.AdamW(model.parameters(), lr=config['lr'], weight_decay=0.01) # What goes in here?

# Declare the decoder. Use the CTC Beam Decoder to decode phonemes
# CTC Beam Decoder Doc: https://github.com/parlance/ctcdecode
decoder = CTCBeamDecoder(LABELS, blank_id=0, beam_width=config["beam_width"], log_probs_input=True) #TODO

scheduler = scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="min", factor = 0.5, patience=4, verbose=True) #TODO

# Mixed Precision, if you need it
scaler = torch.cuda.amp.GradScaler()

## Decode Prediction

In [ ]:
def decode_prediction(output, output_lens, decoder, PHONEME_MAP= LABELS):

    # TODO: look at docs for CTC.decoder and find out what is returned here. Check the shape of output and expected shape in decode.
    beam_results, beam_scores, timesteps, out_lens = decoder.decode(output, seq_lens= output_lens) #lengths - list of lengths

    pred_strings                    = []

    for i in range(output_lens.shape[0]):
        #TODO: Create the prediction from the output of decoder.decode. Don't forget to map it using PHONEMES_MAP.
      out_labels = beam_results[i][0][:out_lens[i][0]]
      pred_strings = pred_strings + [[PHONEME_MAP[n] for n in out_labels]]
    return pred_strings

def calculate_levenshtein(output, label, output_lens, label_lens, decoder, PHONEME_MAP= LABELS): # y - sequence of integers

    dist            = 0
    batch_size      = label.shape[0]

    pred_strings    = decode_prediction(output, output_lens, decoder, PHONEME_MAP)

    for i in range(batch_size):
        # TODO: Get predicted string and label string for each element in the batch
        pred_string = pred_strings[i] #TODO
        label_string = label_string = [PHONEME_MAP[n] for n in label[i]][:len(pred_string)] #TODO
        dist += Levenshtein.distance(pred_string, label_string)

    dist /= batch_size # TODO: Uncomment this, but think about why we are doing this
    return dist

In [ ]:
# test code to check shapes
torch.cuda.empty_cache()
gc.collect()
model.eval()
for i, data in enumerate(val_loader, 0):
    x, y, lx, ly = data
    x, y = x.to(device), y.to(device)
    h, lh = model(x, lx)
    print(h.shape)
    h = torch.permute(h, (1, 0, 2))
    print(h.shape, y.shape)
    loss = criterion(h, y, lh, ly)
    print(loss)

    print(calculate_levenshtein(torch.permute(h, (1,0,2)), y, lx, ly, decoder, LABELS))

    break

torch.Size([64, 734, 41])
torch.Size([734, 64, 41]) torch.Size([64, 265])
tensor(7.6674, device='cuda:0', grad_fn=<MeanBackward0>)
197.890625


## wandb

You will need to fetch your api key from wandb.ai

In [ ]:
import wandb
wandb.login(key="06c8f81427188d28de31ac7bfe0ddeadd005abc3")

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
run = wandb.init(
    name = "submission", ## Wandb creates random run names if you skip this field
    reinit = True, ### Allows reinitalizing runs when you re-run this cell
    # run_id = ### Insert specific run id here if you want to resume a previous run
    # resume = "must" ### You need this to resume previous runs, but comment out reinit = True when using this
    project = "hw3p2-ablations", ### Project should be created in your wandb account
    config = config ### Wandb Config for your run
)

wandb: Currently logged in as: jiayang2 (jgao). Use `wandb login --relogin` to force relogin


# Train Functions

In [ ]:
from tqdm import tqdm

def train_model(model, train_loader, criterion, optimizer):

    model.train()
    batch_bar = tqdm(total=len(train_loader), dynamic_ncols=True, leave=False, position=0, desc='Train')

    total_loss = 0

    for i, data in enumerate(train_loader):
        optimizer.zero_grad()

        x, y, lx, ly = data
        x, y = x.to(device), y.to(device)

        with torch.cuda.amp.autocast():
            h, lh = model(x, lx)
            h = torch.permute(h, (1, 0, 2))
            loss = criterion(h, y, lh, ly)

        total_loss += loss.item()

        batch_bar.set_postfix(
            loss="{:.04f}".format(float(total_loss / (i + 1))),
            lr="{:.06f}".format(float(optimizer.param_groups[0]['lr'])))

        batch_bar.update() # Update tqdm bar

        # Another couple things you need for FP16.
        scaler.scale(loss).backward() # This is a replacement for loss.backward()
        scaler.step(optimizer) # This is a replacement for optimizer.step()
        scaler.update() # This is something added just for FP16

        del x, y, lx, ly, h, lh, loss
        torch.cuda.empty_cache()

    batch_bar.close() # You need this to close the tqdm bar

    return total_loss / len(train_loader)


def validate_model(model, val_loader, decoder, phoneme_map= LABELS):

    model.eval()
    batch_bar = tqdm(total=len(val_loader), dynamic_ncols=True, position=0, leave=False, desc='Val')

    total_loss = 0
    vdist = 0

    for i, data in enumerate(val_loader):

        x, y, lx, ly = data
        x, y = x.to(device), y.to(device)

        with torch.inference_mode():
            h, lh = model(x, lx)
            h = torch.permute(h, (1, 0, 2))
            loss = criterion(h, y, lh, ly)

        total_loss += float(loss)
        vdist += calculate_levenshtein(torch.permute(h, (1, 0, 2)), y, lh, ly, decoder, phoneme_map)

        batch_bar.set_postfix(loss="{:.04f}".format(float(total_loss / (i + 1))), dist="{:.04f}".format(float(vdist / (i + 1))))

        batch_bar.update()

        del x, y, lx, ly, h, lh, loss
        torch.cuda.empty_cache()

    batch_bar.close()
    total_loss = total_loss/len(val_loader)
    val_dist = vdist/len(val_loader)
    return total_loss, val_dist

In [ ]:
# test code to check shapes
torch.cuda.empty_cache()
gc.collect()
model.eval()
for i, data in enumerate(val_loader, 0):
    x, y, lx, ly = data
    x, y = x.to(device), y.to(device)
    h, lh = model(x, lx)
    print(h.shape)
    h = torch.permute(h, (1, 0, 2))
    print(h.shape, y.shape)
    loss = criterion(h, y, lh, ly)
    print(loss)

    print(calculate_levenshtein(torch.permute(h, (1,0,2)), y, lx, ly, decoder, LABELS))

    break

torch.Size([64, 734, 41])
torch.Size([734, 64, 41]) torch.Size([64, 265])
tensor(7.6674, device='cuda:0', grad_fn=<MeanBackward0>)
197.890625


### Training Setup

In [ ]:
def save_model(model, optimizer, scheduler, metric, epoch, path):
    torch.save(
        {'model_state_dict'         : model.state_dict(),
         'optimizer_state_dict'     : optimizer.state_dict(),
         'scheduler_state_dict'     : scheduler.state_dict(),
         metric[0]                  : metric[1],
         'epoch'                    : epoch},
         path
    )

def load_model(path, model, metric= 'valid_acc', optimizer= None, scheduler= None):

    checkpoint = torch.load(path)
    model.load_state_dict(checkpoint['model_state_dict'])

    if optimizer != None:
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    if scheduler != None:
        scheduler.load_state_dict(checkpoint['scheduler_state_dict'])

    epoch   = checkpoint['epoch']
    metric  = checkpoint[metric]

    return [model, optimizer, scheduler, epoch, metric]

In [ ]:
# This is for checkpointing, if you're doing it over multiple sessions

last_epoch_completed = 0
start = last_epoch_completed
end = config["epochs"]
best_lev_dist = float("inf") # if you're restarting from some checkpoint, use what you saw there.
epoch_model_path = '/content/gdrive/MyDrive/Colab Notebooks/11-785/HW3P2/hw3_epoch_model.pth' #TODO set the model path( Optional, you can just store best one. Make sure to make the changes below )
best_model_path = '/content/gdrive/MyDrive/Colab Notebooks/11-785/HW3P2' #TODO set best model path

In [ ]:
torch.cuda.empty_cache()
gc.collect()

#TODO: Please complete the training loop

for epoch in range(0, config['epochs']):

    print("\nEpoch: {}/{}".format(epoch+1, config['epochs']))

    curr_lr = float(optimizer.param_groups[0]['lr']) #TODO

    train_loss              = train_model(model, train_loader, criterion, optimizer) #TODO
    valid_loss, valid_dist  = validate_model(model, val_loader, decoder, LABELS) #TODO
    scheduler.step(valid_dist)

    print("\tTrain Loss {:.04f}\t Learning Rate {:.07f}".format(train_loss, curr_lr))
    print("\tVal Dist {:.04f}%\t Val Loss {:.04f}".format(valid_dist, valid_loss))


    # wandb.log({
    #     'train_loss': train_loss,
    #     'valid_dist': valid_dist,
    #     'valid_loss': valid_loss,
    #     'lr'        : curr_lr
    # })

    save_model(model, optimizer, scheduler, ['valid_dist', valid_dist], epoch, epoch_model_path)
    # wandb.save(epoch_model_path)
    print("Saved epoch model")

    # if valid_dist <= best_lev_dist:
    #     best_lev_dist = valid_dist
    #     torch.save({'model_state_dict'         : model.state_dict(),
    #                 'optimizer_state_dict'     : optimizer.state_dict(),
    #                 'scheduler_state_dict'     : scheduler.state_dict(),
    #                 'val_dist'                 : valid_dist}, f'{best_model_path}/model-pLSTM_{epoch}.pth')
        # save_model(model, optimizer, scheduler, ['valid_dist', valid_dist], epoch, best_model_path)
        # wandb.save(best_model_path)
#         print("Saved best model")
#       # You may find it interesting to exlplore Wandb Artifcats to version your models
# run.finish()


Epoch: 1/60


	Train Loss 0.6062	 Learning Rate 0.0020000
	Val Dist 9.8679%	 Val Loss 0.3906


RuntimeError: ignored

# Generate Predictions and Submit to Kaggle

In [ ]:
checkpoint = torch.load('/content/gdrive/MyDrive/Colab Notebooks/DL/hw3p2/model-pLSTM_80.pth')

model = ASRModel(
    input_size  = 27,
    embed_size  = 300, #TODO
    output_size = len(PHONEMES)
).to(device)
model.load_state_dict(checkpoint['model_state_dict'])

optimizer =  torch.optim.AdamW(model.parameters(), lr=config["lr"], weight_decay=5e-6)
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
# optimizer =  torch.optim.AdamW(model.parameters(), lr=2e-4, weight_decay=5e-6) # What goes in here?

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="min", factor = 0.5, patience=4, verbose=True)
scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
val_dist = checkpoint['val_dist']

In [ ]:
#TODO: Make predictions

# Follow the steps below:
# 1. Create a new object for CTCBeamDecoder with larger (why?) number of beams
# 2. Get prediction string by decoding the results of the beam decoder

TEST_BEAM_WIDTH = 200 #TODO

test_decoder    = CTCBeamDecoder(LABELS, blank_id=0, beam_width=TEST_BEAM_WIDTH, log_probs_input=True) #TODO
results = []

model.eval()
print("Testing")
for data in tqdm(test_loader):

    x, lx   = data
    x       = x.to(device)

    with torch.no_grad():
        h, lh = model(x, lx)

    prediction_string = decode_prediction(h, lh, test_decoder, PHONEME_MAP= LABELS)# TODO call decode_prediction
    #TODO save the output in results array.
    for s in prediction_string:
      results.append("".join(s))

    del x, lx, h, lh
    torch.cuda.empty_cache()

In [ ]:
data_dir = '/content/11-785-s23-hw3p2' + "/test-clean/transcript/random_submission.csv"
df = pd.read_csv(data_dir)
df.label = results
df.to_csv('submission.csv', index = False)

!kaggle competitions submit -c automatic-speech-recognition-asr-slack-kaggle -f submission.csv -m "Message"
